In [1]:
# import the Dash package
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash


In [2]:
from data_processor import dataprocessor
import plot_maker
from constants import DESCRIPTION_URL_DICTIONARY
import pandas as pd
from datetime import datetime, date, timedelta
from dash import dcc, html, Input, Output, no_update

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([dcc.DatePickerSingle(id='date-picker-single',
                                   min_date_allowed=date(2012, 10, 1),
                                   max_date_allowed=date(2017, 10, 30),
                                   initial_visible_month=date(2015, 8, 5),
                                   date=date(2015, 8, 25),
                                   style={'float': 'left', 'margin': 'auto'}
                                   ),
              html.Br(),
              dcc.RadioItems(options=[{'label': 'per hour', 'value': 'hourly'},
                                      {'label': 'average', 'value': 'daily_average'}
                                      ],
                             value='hourly',
                             id='data_type',
                             style={'float': 'left', 'margin': 'auto'},
                             )],
             className='row'),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Div(id='slider_container', children=[dcc.Slider(0, 23, 1, id='time-select',
                                                         marks={
                                                             val: (f'{val}am' if val < 12 else f'{val%12}pm') for val in range(24)},
                                                         value=12)]),
    dcc.Graph(id='plot', clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip")
])


@app.callback(
    Output('slider_container', 'style'),
    Input('data_type', 'value'))
def set_cities_options(option):
    if option == 'hourly':
        return {'display': 'block'}
    else:
        return {'display': 'none'}


@app.callback(
    Output('plot', 'figure'),
    Input('date-picker-single', 'date'),
    Input('time-select', 'value'),
    Input('data_type', 'value')
)
def update_figure(date, time, datatype):
    if datatype == 'hourly':
        datetime_datetime = datetime.fromisoformat(
            date) + timedelta(hours=time)
        datetime_datetime = pd.to_datetime(datetime_datetime)
    else:
        datetime_datetime = datetime.fromisoformat(date).date()

    data_in_time = dataprocessor.get_organized_wind_data_in_time(
        datetime_datetime, datatype)
    fig = plot_maker.PlotMaker(data_in_time).plot_maker()
    fig.update_traces(hoverinfo="none", hovertemplate=None)
    return fig


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input('plot', "hoverData"),
    Input('date-picker-single', 'date'),
    Input('time-select', 'value'),
    Input('data_type', 'value')
)
def update_hover(hoverData, date, time, datatype):
    if hoverData is None:
        return False, no_update, no_update
    pt_lat = hoverData["points"][0]['lat']
    pt_lon = hoverData["points"][0]['lon']
    bbox = hoverData["points"][0]["bbox"]

    if datatype == 'hourly':
        datetime_datetime = datetime.fromisoformat(
            date) + timedelta(hours=time)
        datetime_datetime = pd.to_datetime(datetime_datetime)
    else:
        datetime_datetime = datetime.fromisoformat(date).date()

    city, temp, desc = dataprocessor.get_hover_data_in_time(
        pt_lat, pt_lon, datetime_datetime, datatype)
    img_src = DESCRIPTION_URL_DICTIONARY[desc]

    if len(desc) > 300:
        desc = desc[:100] + '...'

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{city}", style={"color": "darkblue",
                    "overflow-wrap": "break-word"}),
            html.P(f"{temp:.1f}"u'\N{DEGREE SIGN}F'),
            html.P(f'{desc}'),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children


app.run_server(mode='inline', port=8062)


Dash is running on http://127.0.0.1:8062/

